<a href="https://colab.research.google.com/github/ThangLC304/ThangLC304/blob/main/CourseraMonth3Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import packages

In [4]:
from google.colab import files

import networkx as nx
from networkx.algorithms.euler import eulerian_circuit
from networkx.algorithms.euler import is_eulerian
from networkx.algorithms.euler import has_eulerian_path
from networkx.algorithms.euler import eulerian_path
from networkx.algorithms.euler import eulerize

import itertools
from itertools import product
from itertools import islice
from itertools import combinations
from more_itertools import sliced

import pandas as pd

import re 

import shutil

import sys

import copy

import time

import math

import numpy as np

from operator import itemgetter

from functools import lru_cache

import subprocess
import pkg_resources

required = {'gTTS'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio #Import Audio method from IPython's Display Class

from google.colab import drive
drive._mount('/content/drive')

ValueError: ignored

## Part 1.2: Global Alignment Problem: Find a highest-scoring alignment of two strings as defined by a scoring matrix.

Input: Two strings and a scoring matrix Score.

Output: An alignment of the strings whose alignment score (as defined by Score) is maximized over all alignments of the strings.

To solve the Global Alignment Problem, we still must find a longest path in the alignment graph after updating the edge weights to reflect the values in the scoring matrix. Recalling that deletions correspond to vertical edges (↓), insertions correspond to horizontal edges (→), and matches/mismatches correspond to diagonal edges (↘/↘), we obtain the following recurrence for si, j, the length of a longest path from (0, 0) to (i, j):

Figure

<img src="https://bioinformaticsalgorithms.com/images/Alignment/global_alignment_recurrence_2.png" width = 300>

<img src="https://bioinformaticsalgorithms.com/images/Alignment/colored_alignment_graph.png" width = 300>

### Problem 1: Solve the Global Alignment Problem.

Input: Two protein strings written in the single-letter amino acid alphabet.

Output: The maximum alignment score of these strings followed by an alignment achieving this maximum score. Use the BLOSUM62 scoring matrix for matches and mismatches as well as the indel penalty σ = 5.'

```
Sample Input:
PLEASANTLY
MEANLY
Sample Output:
8
PLEASANTLY
-MEA--N-LY
```

In [ ]:
filename = "BlosumScore.txt"
rootpath = "/content/drive/MyDrive/Coursera/Month3/Week2/"
filepath = rootpath+filename
with open(filepath, "r") as file:
  raw = file.read().splitlines()

raw

In [ ]:
BlosumScoreMap = [x.split() for x in raw]
print(BlosumScoreMap)

In [ ]:
BlosumScoreMapValue = [x[1:] for x in BlosumScoreMap[1:]]
print(BlosumScoreMapValue)

#### Blosum map

In [ ]:
df = pd.DataFrame(BlosumScoreMapValue, columns = BlosumScoreMap[0], index = BlosumScoreMap[0])
df

In [ ]:
#Call cell value by row and col names
target_cell = df.loc['A','A'] #[row_label, col_label]
target_cell

In [ ]:
#Call cell value by row and col index
target_cell = df.iloc[0,0]
target_cell

In [ ]:
def ScoreMapGlobal(v, w, datamap):
  scores = [[0] * (len(w)+1) for _ in range(len(v)+1)]
  for i in range(1, len(v)+1):
    for j in range(1, len(w)+1):
      scores[i][j] = datamap.loc[v[i-1], w[j-1]]
  return scores

In [ ]:
seq1 = "PLEASANTLY"
seq2 = "MMMMEANLY"
ScoreMapGlobal(seq1, seq2, df)



```
        M   E   A   N   L   Y
    0→ -5→-10→-15→-20→-25→-30
    ↓ ↘   ↘                   
P  -5  -2  -6→-11→-16→-21→-26
    ↓ ↘   ↘   ↘       ⇘       
L -10  -3  -5  -7→-12 -12→-17
    ↓   ↓ ⇘       ↘       ↘   
E -15  -8   2→ -3  -7→-12 -14
    ↓   ↓   ↓ ⇘               
A -20 -13  -3   6→  1→ -4→ -9
    ↓   ↓   ↓   ↓ ↘           
S -25 -18  -8   1   7→  2→ -3
    ↓   ↓   ↓   ↓   ↓ ↘       
A -30 -23 -13  -4   2   6→  1
    ↓   ↓   ↓   ↓ ⇘     ↓ ↘   
N -35 -28 -18  -9   2   1   4
    ↓   ↓   ↓   ↓   ↓ ↘     ↓ 
T -40 -33 -23 -14  -3   1  -1
    ↓   ↓   ↓   ↓   ↓ ⇘   ↘   
L -45 -38 -28 -19  -8   1   0
    ↓   ↓   ↓   ↓   ↓   ↓ ⇘   
Y -50 -43 -33 -24 -13  -4   8
```



In [ ]:
def BackTrackWithScore(v, w):
  Backtrack = [['0'] * (len(w)+1) for _ in range(len(v)+1)]
  scores = [[0] * (len(w)+1) for _ in range(len(v)+1)]
  for i in range(0, len(v)+1):
    scores[i][0] = (-5)*i
  for j in range(0, len(w)+1):
    scores[0][j] = (-5)*j
  for i in range(1, len(v)+1):
    for j in range(1, len(w)+1):
      # point = 0
      # if v[i-1] == w[j-1]:
      #   point = int(ScoreMap(v, w)[i][j])
      #   print(i, j, point)
      point = int(ScoreMapGlobal(v, w, df)[i][j])
      scores[i][j] = max(scores[i-1][j] - 5, scores[i][j-1] - 5, scores[i-1][j-1] + point)
      if scores[i][j] == (scores[i-1][j] - 5):
        Backtrack[i][j] = '↓'
      elif scores[i][j] == (scores[i][j-1] -5):
        Backtrack[i][j] = '→'
      elif scores[i][j] == (scores[i-1][j-1] + point):
        Backtrack[i][j] = '↘'
  return Backtrack, scores

def OutputLCS(backtrack, v, w):
  i = len(seq1)
  j = len(seq2)
  LCS_1 = ""
  LCS_2 = ""
  while i >=0 and j >= 0:
    if (i-1) >= 0:
      add_1 = v[i-1]
    else:
      if j>0: 
        add_1 = "-"
      else: 
        add_1 = ""

    if (j-1) >= 0:
      add_2 = w[j-1]
    else:
      if i>0: 
        add_2 = "-"
      else: 
        add_2 = ""

    if backtrack[i][j] == "↘":
      LCS_1 += add_1
      LCS_2 += add_2
      i -= 1
      j -= 1
    elif backtrack[i][j] == "↓":
      LCS_2 += "-"
      LCS_1 += add_1
      i -= 1
    elif backtrack[i][j] == "→":
      LCS_2 += "-"
      LCS_1 += "-"
      j -= 1
    else:
      LCS_1 += add_1
      LCS_2 += add_2
      j -= 1
      i -= 1
  return LCS_1[::-1], LCS_2[::-1]

backtrack, score = BackTrackWithScore(seq1, seq2)
score

In [ ]:
backtrack

In [ ]:
def display_path(v, w):
  backtrack_input, _ = BackTrackWithScore(v, w)
  backtrack_input = backtrack_input[1:]
  backtrack_input = [x[1:] for x in backtrack_input]
  output_df = pd.DataFrame(backtrack_input, index = [x for x in v], columns = [x for x in w])
  return output_df

display_path(seq1, seq2)

In [ ]:
output_1, output_2 = OutputLCS(backtrack, seq1, seq2)
print(score[-1][-1])
print(output_1)
print(output_2)
# 8
# PLEASANTLY
# -MEA--N-LY

#### Sum up the challenge

In [ ]:
filename = "BlosumScore.txt"
rootpath = "/content/drive/MyDrive/Coursera/Month3/Week2/"
filepath = rootpath+filename
with open(filepath, "r") as file:
  raw = file.read().splitlines()

BlosumScoreMap = [x.split() for x in raw]
BlosumScoreMapValue = [x[1:] for x in BlosumScoreMap[1:]]
df = pd.DataFrame(BlosumScoreMapValue, columns = BlosumScoreMap[0], index = BlosumScoreMap[0])

def GlobalAlignment(seq1, seq2):
  def ScoreMap(v, w):
    scores = [[0] * (len(w)+1) for _ in range(len(v)+1)]
    for i in range(1, len(v)+1):
      for j in range(1, len(w)+1):
        scores[i][j] = df.loc[v[i-1], w[j-1]]
    return scores

  def BackTrackWithScore(v, w, score_map):
    Backtrack = [['0'] * (len(w)+1) for _ in range(len(v)+1)]
    scores = [[0] * (len(w)+1) for _ in range(len(v)+1)]
    for i in range(0, len(v)+1):
      scores[i][0] = (-5)*i
    for j in range(0, len(w)+1):
      scores[0][j] = (-5)*j
    for i in range(1, len(v)+1):
      for j in range(1, len(w)+1):
        # point = 0
        # if v[i-1] == w[j-1]:
        #   point = int(ScoreMap(v, w)[i][j])
        #   print(i, j, point)
        point = int(score_map[i][j])
        scores[i][j] = max(scores[i-1][j] - 5, scores[i][j-1] - 5, scores[i-1][j-1] + point)
        if scores[i][j] == (scores[i-1][j] - 5):
          Backtrack[i][j] = '↓'
        elif scores[i][j] == (scores[i][j-1] -5):
          Backtrack[i][j] = '→'
        elif scores[i][j] == (scores[i-1][j-1] + point):
          Backtrack[i][j] = '↘'
    return Backtrack, scores

  def OutputLCS(backtrack, v, w):
    i = len(seq1)
    j = len(seq2)
    LCS_1 = ""
    LCS_2 = ""
    while i >=0 and j >= 0:
      if (i-1) >= 0:
        add_1 = v[i-1]
      else:
        if j>0: 
          add_1 = "-"
        else: 
          add_1 = ""

      if (j-1) >= 0:
        add_2 = w[j-1]
      else:
        if i>0: 
          add_2 = "-"
        else: 
          add_2 = ""

      if backtrack[i][j] == "↘":
        LCS_1 += add_1
        LCS_2 += add_2
        i -= 1
        j -= 1
      elif backtrack[i][j] == "↓":
        LCS_2 += "-"
        LCS_1 += add_1
        i -= 1
      elif backtrack[i][j] == "→":
        LCS_2 += add_2
        LCS_1 += "-"
        j -= 1
      else:
        LCS_1 += add_1
        LCS_2 += add_2
        j -= 1
        i -= 1
    return LCS_1[::-1], LCS_2[::-1]

  def display_path(v, w, score_map):
    backtrack_input, _ = BackTrackWithScore(v, w, score_map)
    backtrack_input = backtrack_input[1:]
    backtrack_input = [x[1:] for x in backtrack_input]
    output_df = pd.DataFrame(backtrack_input, index = [x for x in v], columns = [x for x in w])
    return output_df
  score_map = ScoreMap(seq1, seq2)
  print(display_path(seq1, seq2, score_map))
  backtrack, score = BackTrackWithScore(seq1, seq2, score_map)
  output_1, output_2 = OutputLCS(backtrack, seq1, seq2)
  print(score[-1][-1])
  print(output_1)
  print(output_2)
  return score[-1][-1], output_1, output_2


#### Run extradataset

In [ ]:
filename = "extradataset_1_2_1.txt"
rootpath = "/content/drive/MyDrive/Coursera/Month3/Week2/"
filepath = rootpath+filename
with open(filepath, mode='r', encoding="utf-8-sig") as file:
  raw = file.read().splitlines()

seq1 = raw[0]
seq2 = raw[1]
score_test = raw[2]
output_1_test = raw[3]
output_2_test = raw[4]

print(seq1)
print(seq2)
print(score_test)
print(output_1_test)
print(output_2_test)

In [ ]:
score, output_1, output_2 = GlobalAlignment(seq1, seq2)

In [ ]:
if int(score) == int(score_test): print("Score correct")
else: print("Score incorrect")
if output_1 == output_1_test: print("Output 1 correct")
else: print("Output 1 incorrect")
if output_2 == output_2_test: print("Output 2 correct")
else: print("Output 2 incorrect")

#### Run test dataset

In [ ]:
filename = "dataset_247_3.txt"
rootpath = "/content/drive/MyDrive/Coursera/Month3/Week2/"
filepath = rootpath+filename
with open(filepath, mode='r', encoding="utf-8-sig") as file:
  raw = file.read().splitlines()

seq1 = raw[0]
seq2 = raw[1]

GlobalAlignment(seq1, seq2)

### Problem 2: Solve the Local Alignment Problem.

Input: Two protein strings written in the single-letter amino acid alphabet.

Output: The maximum score of a local alignment of the strings, followed by a local alignment of these strings achieving the maximum score. Use the PAM250 scoring matrix for matches and mismatches as well as the indel penalty σ = 5.

```
Sample Input:

MEANLY
PENALTY

Sample Output:

15
EANL-Y
ENALTY
```



#### PAM250 score map

In [ ]:
filename = "PAM250.txt"
rootpath = "/content/drive/MyDrive/Coursera/Month3/Week2/"
filepath = rootpath+filename
with open(filepath, "r") as file:
  raw = file.read().splitlines()

raw

In [ ]:
PAM250Map = [x.split() for x in raw]
print(PAM250Map)

In [ ]:
PAM250MapValue = [x[1:] for x in PAM250Map[1:]]
print(PAM250MapValue)

In [ ]:
PAM250 = pd.DataFrame(PAM250MapValue, columns = PAM250Map[0], index = PAM250Map[0])
PAM250

#### def LocalAlignment(seq1, seq2)

In [ ]:
def LocalAlignment(seq1, seq2):
  def ScoreMap(v, w):
    scores = [[0] * (len(w)+1) for _ in range(len(v)+1)]
    for i in range(1, len(v)+1):
      for j in range(1, len(w)+1):
        scores[i][j] = PAM250.loc[v[i-1], w[j-1]]
    return scores

  def BackTrackWithScore(v, w, score_map):
    Backtrack = [['0'] * (len(w)+1) for _ in range(len(v)+1)]
    scores = [[0] * (len(w)+1) for _ in range(len(v)+1)]
    max_score = 0
    location = [-1, -1]
    for i in range(1, len(v)+1):
      for j in range(1, len(w)+1):
        # point = 0
        # if v[i-1] == w[j-1]:
        #   point = int(ScoreMap(v, w)[i][j])
        #   print(i, j, point)
        point = 0
        if v[i-1] == w[j-1]:
          point = int(score_map[i][j])
        scores[i][j] = max(0, scores[i-1][j] - 5, scores[i][j-1] - 5, scores[i-1][j-1] + point)
        if scores[i][j] > max_score:
          max_score = scores[i][j]
          location = [i, j]
        if scores[i][j] == 0:
          Backtrack[i][j] = '.'
        elif scores[i][j] == (scores[i][j-1] -5):
          Backtrack[i][j] = '→'
        elif scores[i][j] == (scores[i-1][j] - 5):
          Backtrack[i][j] = '↓'
        elif scores[i][j] == (scores[i-1][j-1] + point):
          Backtrack[i][j] = '↘'
    return Backtrack, scores, max_score, location

  def OutputLCS(backtrack, v, w, location):
    i = location[0]
    j = location[1]
    LCS_1 = ""
    LCS_2 = ""
    while i >=0 and j >= 0:
      if (i-1) >= 0:
        add_1 = v[i-1]
      else:
        if j>0: 
          add_1 = "-"
        else: 
          add_1 = ""

      if (j-1) >= 0:
        add_2 = w[j-1]
      else:
        if i>0: 
          add_2 = "-"
        else: 
          add_2 = ""

      if backtrack[i][j] == ".":
        break
      elif backtrack[i][j] == "↓":
        LCS_2 += "-"
        LCS_1 += add_1
        i -= 1
      elif backtrack[i][j] == "→":
        LCS_2 += add_2
        LCS_1 += "-"
        j -= 1
      elif backtrack[i][j] == "↘":
        LCS_1 += add_1
        LCS_2 += add_2
        i -= 1
        j -= 1
      else:
        LCS_1 += add_1
        LCS_2 += add_2
        j -= 1
        i -= 1
    return LCS_1[::-1], LCS_2[::-1]

  def display_path(v, w, score_map):
    backtrack_input, _, __, ___ = BackTrackWithScore(v, w, score_map)
    backtrack_input = backtrack_input[1:]
    backtrack_input = [x[1:] for x in backtrack_input]
    output_df = pd.DataFrame(backtrack_input, index = [x for x in v], columns = [x for x in w])
    return output_df

  score_map = ScoreMap(seq1, seq2)
  print(display_path(seq1, seq2, score_map))
  backtrack, scores, max_score, location = BackTrackWithScore(seq1, seq2, score_map)
  output_1, output_2 = OutputLCS(backtrack, seq1, seq2, location)
  # print(score[-1][-1])

  print("Exported Score map to temp_score_map.txt: ")
  filename_ = "temp_score_map.txt"
  rootpath_ = "/content/drive/MyDrive/Coursera/Month3/Week2/"
  filepath_ = rootpath_+filename_
  with open(filepath_, mode='w', encoding="utf-8-sig") as file:
    for x in scores:
      x = ', '.join(str(e) for e in x)
      file.write(x+"\n")

  # temp_score_map = pd.DataFrame(scores, columns = list(seq2), index = list(seq1))
  # filename_ = "temp_score_map.csv"
  # rootpath_ = "/content/drive/MyDrive/Coursera/Month3/Week2/"
  # filepath_ = rootpath_+filename_
  # temp_score_map.to_csv(filepath_)
                         
  print("Output 1: ",output_1)
  print("Output 2: ",output_2)
  return max_score, output_1, output_2

#### Run Example dataset

In [ ]:
seq1 = "MEANLY"
seq2 = "PENALTY"
ScoreMapGlobal(seq1, seq2, PAM250)

In [ ]:
LocalAlignment(seq1, seq2)

```
      M  E  A  N  L   Y
  [0, 0, 0, 0, 0, 0,  0]
P [0, 0, 0, 1, 0, 0,  0]
E [0, 0, 4, 0, 2, 0,  0]
N [0, 0, 1, 4, 2, 0,  0]
A [0, 0, 0, 3, 4, 0,  0]
L [0, 4, 0, 0, 0, 10, 5]
T [0, 0, 4, 1, 0, 5,  7]
Y [0, 0, 0, 1, 0, 0,  15]
```



#### Run Extradataset

In [ ]:
filename = "extradataset_1_2_2.txt"
rootpath = "/content/drive/MyDrive/Coursera/Month3/Week2/"
filepath = rootpath+filename
with open(filepath, mode='r', encoding="utf-8-sig") as file:
  raw = file.read().splitlines()

seq1 = raw[0]
seq2 = raw[1]
score_test = raw[2]
output_1_test = raw[3]
output_2_test = raw[4]

print(seq1)
print(seq2)
print(score_test)
print(output_1_test)
print(output_2_test)
print(len(output_1_test))

In [ ]:
score, output_1, output_2 = LocalAlignment(seq1, seq2)

In [ ]:
len(output_1)

In [ ]:
if int(score) == int(score_test): print("Score correct")
else: print("Score incorrect")
if output_1 == output_1_test: print("Output 1 correct")
else: print("Output 1 incorrect")
if output_2 == output_2_test: print("Output 2 correct")
else: print("Output 2 incorrect")